In [1]:
import sys
import os

# Fix the module path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Imports
from src.clients.solana.meteora import SolanaMeteoraClient
from src.utils.analyzer import Analyzer

# Initialize client
client = SolanaMeteoraClient()



Setting API URL: https://dlmm-api.meteora.ag
Setting RPC: https://api.mainnet-beta.solana.com
Creating session...
Session headers set: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/json', 'Connection': 'keep-alive', 'Content-type': 'application/json', 'rpc': 'https://api.mainnet-beta.solana.com'}


In [2]:
pairs = client.fetch_liquidity_pools()


min_tvl = 100000  
min_volume = 5000 
filtered_pairs = client.filter_pairs(pairs, min_tvl=min_tvl, min_volume=min_volume)

print(f"Total pairs fetched: {len(pairs)}")
print(f"Filtered pairs (TVL >= {min_tvl}, Volume >= {min_volume}): {len(filtered_pairs)}")
for pair in filtered_pairs[:5]:
    print(pair)



Fetching pools from https://dlmm-api.meteora.ag/pair/all
Status code: 200
Total pools in response: 69014
Sample pool: {'address': 'BqBQY7dTuNSAh3Ngf1xSARhVc5VZke8LPhCxP3G6f4hz', 'name': 'JAK-SOL', 'mint_x': 'FBbnzHwJ1WHYwP425cqMNb2t7o7sm6AXQjsQ3sZRpump', 'mint_y': 'So11111111111111111111111111111111111111112', 'reserve_x': '6BjWUGZPqsCq3KjF8yZvh6f165UBKWR7DRhB58gZp6yd', 'reserve_y': 'BKD5HqHcPTvvbmGJsdjrzcLWzdcn11zmxnCBsnRxjgnB', 'reserve_x_amount': 70, 'reserve_y_amount': 34, 'bin_step': 200, 'base_fee_percentage': '1', 'max_fee_percentage': '7.75', 'protocol_fee_percentage': '5', 'liquidity': '0.000005658031934692075', 'reward_mint_x': '11111111111111111111111111111111', 'reward_mint_y': '11111111111111111111111111111111', 'fees_24h': 0.0, 'today_fees': 0.0, 'trade_volume_24h': 0.0, 'cumulative_trade_volume': '566.5800', 'cumulative_fee_volume': '18.0300', 'current_price': 2.145688522810845e-05, 'apr': 0.0, 'apy': 0.0, 'farm_apr': 0.0, 'farm_apy': 0.0, 'hide': False, 'is_blacklisted'

In [ ]:
print(f"Length of pairs: {len(pairs)}")
for pair in filtered_pairs[:5]:
    print(pair)

# Test price bars
price_bars = client.get_price_bars("JUPyiwrYJFskUPiHa7hkeR8VUtAeFoSYbKedZNsDvCN", "USDC")
print(f"Price bars: \n{price_bars.data.head()}")

# Test positions
positions = client.get_open_positions("Yj7SzJwGkHuUKBfFytp8TPfj997ntSicCCuJLiB39kE")
print(f"Positions: {len(positions)}")

Length of pairs: 69014
LiquidityPair(address='CLnyJa5U2Ysgg9C53UpPN4wNWerF3gWppu4fSzTYBWYx', token0_symbol='GIGA', token0_address='63LfDmNb3MQ8mw9MtZ2To9bEA2M71kZUUGq5tiJxcqj9', token1_symbol='FDUSD', token1_address='9zNQRsGLjNKwCUU5Gq5LR8beUCPzQMVMqKAi3SSZh54u', tvl=961997.9266718655, volume=54195.031568625796)
LiquidityPair(address='2QrWsSWrGvoAqkDC5XSGqjS752RWLaopqAaGrbugSxBL', token0_symbol='JUP', token0_address='JUPyiwrYJFskUPiHa7hkeR8VUtAeFoSYbKedZNsDvCN', token1_symbol='JitoSOL', token1_address='J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn', tvl=1580095.221746826, volume=120378.77476799244)
LiquidityPair(address='DYmirDZoK5WW63HGLWLRyKfkGMKknGhzqts98gPVQ1BX', token0_symbol='WBTC', token0_address='3NZ9JMVBmGAqocybic2c7LQCJScmgsAZ6vQqTDzcqmJh', token1_symbol='WETH', token1_address='7vfCXTUXx5WJV5JADk17DUJ4ksgau7utNKj4b963voxs', tvl=344596.812681484, volume=27877.39520798705)
LiquidityPair(address='oD3ZbhsUCX2VeutNyqVtmS4BCoezDwsbgN1WppHL2rL', token0_symbol='JLP', token0_address='2

/Users/nicolaschiavo/Dev/internship/dex_analyzer/src/clients/solana/meteora.py:109: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "timestamp": pd.date_range(start=start_time, periods=168, freq="1H"),


Positions fetch failed: 404 Client Error: Not Found for url: https://dlmm-api.meteora.ag/dlmm/get-all-lb-pair-positions-by-user
Positions: 0


In [ ]:
# Compute correlation
corr_matrix = Analyzer.compute_correlation_matrix(price_bars)
print("\nCorrelation Matrix:")
print(corr_matrix)

# Check positions and notify
address = "6oFRER5LAnVG9EQPyujKKXqiY7kJwQfJmdqZUXRwo9qK"
positions = client.get_open_positions(address)
notifier = Notifier("YOUR_BOT_TOKEN", "YOUR_CHAT_ID")

for pos in positions:
    # Use the latest price from a relevant pair (simplified)
    current_price = price_bars[0].data["close"].iloc[-1]
    if pos.lower_bound and pos.upper_bound:  # If bounds are available
        if current_price < pos.lower_bound or current_price > pos.upper_bound:
            message = f"Alert: {pos.address} price {current_price} outside [{pos.lower_bound}, {pos.upper_bound}]"
            # await notifier.send_alert(message)  # Uncomment with async setup
            print(message)